The section consists of various section of geo analysis of data
## **Content**

1. [Importing Libraries](#1)
2. [Loading Data](#2)
3. [Preprocessing Data](#3)
4. [Data Viualization](#4)
5. [Tweet Text Analysis](#5)
6. [Sentiment analysis](#6) 

<a id="1"></a>
## Importing Libraries

In [ ]:
# !pip3 install -U pip
# !pip3 install -r requirements.txt
# !sudo apt install python3-dev
# !sudo apt-get install libproj-dev proj-data proj-bin  
# !sudo apt-get install libgeos-dev  
# !sudo apt-get install -y python3-pyproj

In [ ]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from collections import Counter
from datetime import date
import os
import datetime
from sklearn import preprocessing
import json
import seaborn as sns
import geopandas as gpd
from nltk.corpus import stopwords
import geoplot
from geopy import Nominatim
import folium
import mapclassify
import requests
import re
from bs4 import BeautifulSoup
import plotly.express as px 
import plotly.graph_objs as go 
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from mpl_toolkits.axes_grid1 import make_axes_locatable
from folium.plugins import HeatMapWithTime, TimestampedGeoJson
import numpy as np
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from iso3166 import countries
import calmap
from geopy.geocoders import Nominatim
from tqdm import tqdm
import country_converter as coco
import time
import pymongo
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import matplotlib.ticker as ticker
from similarity.cosine import Cosine
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import Binarizer, StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
from sklearn import ensemble


# For Parallel Processing
import swifter

from pandarallel import pandarallel

pandarallel.initialize()

import matplotlib.style as style 
style.use('fivethirtyeight')

In [ ]:
np.random.seed(sum(map(ord, 'calmap')))

In [ ]:
mongo_client = pymongo.MongoClient()
mongo_db = mongo_client["twitter"]
mongo_collection = mongo_db["tweets"]

<a id="2"></a>

## Loading Data

In [ ]:
# Load world cities dataset
cities = pd.read_csv('worldcities.csv')
print(cities.shape)

In [ ]:
# Load tweets dataset from MongoDB
# full_dataframe = pd.DataFrame(mongo_collection.find()[:50000])

# Load old tweets dataset from csv file
# full_dataframe = pd.read_csv("All_Tweets.csv")
full_dataframe = pd.read_csv("covid19_tweets.csv")
# full_dataframe = full_dataframe[["Text"]]
# full_dataframe.columns = ["text"]

full_dataframe.shape

In [ ]:
full_dataframe.head()

In [ ]:
def filter_tweet(tweet):
    new_tweet = {}
    new_tweet['user_name'] = tweet['user']['screen_name']
    new_tweet['user_location'] = tweet['user']['location']
    new_tweet['hashtags'] = tweet['entities']['hashtags']
    new_tweet['is_retweet'] = tweet['retweeted']
    new_tweet['source'] = tweet['source']
    new_tweet['user_description'] = tweet['user']['description']
    new_tweet['user_created'] = tweet['user']['created_at']
    new_tweet['user_followers'] = tweet['user']['followers_count']
    new_tweet['user_friends'] = tweet['user']['friends_count']
    new_tweet['user_favourites'] = tweet['user']['favourites_count']
    new_tweet['user_verified'] = tweet['user']['verified']
    new_tweet['date'] = tweet['created_at']
    new_tweet['text'] = tweet['full_text']
    new_tweet['tweet_id'] = tweet['id']
    if tweet['coordinates']:
        new_tweet['lat'] =  tweet['coordinates']['coordinates'][0]  
        new_tweet['long'] = tweet['coordinates']['coordinates'][1]  
    else:
        new_tweet['lat'] = 'None'
        new_tweet['long'] = 'None'
    new_tweet['geo_enabled'] = tweet['user']['geo_enabled']
    new_tweet['tweet_place'] = tweet['place']
    
    return new_tweet



# rows = [filter_tweet(row) for index, row in full_dataframe.iterrows()]
# full_dataframe = pd.DataFrame(rows)
# df = full_dataframe.drop_duplicates(subset='tweet_id', keep="last")

df = full_dataframe.drop_duplicates(subset='text', keep="last")

del full_dataframe

df.reset_index(inplace=True)
print(df.shape)
df.head()

In [ ]:
row_num, col_num = df.shape
print(f'There are {row_num} rows and {col_num} columns')

In [ ]:
df.info()

In [ ]:
df.describe()

<a id="3"></a>
## Preprocessing Data

In [ ]:
import re, string

def clean_tweets(text):
    '''
    Utility function to clean tweet text by removing links, special characters ..etc
    using simple regex statements.
    '''
    
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", str(text)).split())
    # Remove RT
    text = re.sub("RT @[\w]*:","", text)
    # Remove citations
    text = re.sub("@[\w]*:","", text)
    # Remove hyperlinks
    text = re.sub("https?://[A-Za-z0-9./]","", text)
    text = re.sub("https?:\/\/.*\/\w*","", text)
    # Remove linebreak, tab, return
    text = re.sub('[\n\t\r]+', ' ', text)
    # Remove tickers
    text = re.sub('\$\w*', '', text)
    # Remove punctuation
    text = re.sub('[' + string.punctuation + ']+', '', text)
    # Remove quotes
    text = re.sub('\&*[amp]*\;|gt+', '', text)
    # Remove via with blank
    text = re.sub('via+\s', '', text)
    # Remove multiple whitespace
    text = re.sub('\s+\s+', ' ', text)
    # Remove multiple whitespace
    text = re.sub('\s+\s+', ' ', text)
    # Remove HashTags 
    text = re.sub('\#+[\w_]+[\w\'_\-]*[\w_]+', ' ', text)
    # Remove Smileys
    text = re.sub('[:=]+(|o|O| )+[D\)\]]+[\(\[]+[pP]+[doO/\\]+[\(\[]+(\^_\^|)', ' ', text)

    return text.lower().strip()


def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    if type(tweet) is type(None):
        return "None"
    
    # create TextBlob object of passed tweet text
    analysis = TextBlob(tweet)
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'


def regularExpression(textToFilter):
    filteredTweet = []
    retweetPattern = 'RT|@RT'
    urlPattern = 'https://[a-zA-Z0-9+&@#/%?=~_|!:,.;]*'

    for textLine in textToFilter:
        tweet = re.sub(retweetPattern,'',textLine)
        tweet = re.sub(urlPattern,'',tweet)
        filteredTweet.append(tweet)
    return filteredTweet


def nltkTokenizer(textToTokenize):
    filteredSentence = []
    usersPattern = re.compile('@[a-zA-Z0-9]*',re.UNICODE)
    hashtagPattern = re.compile('#[a-zA-Z0-9]*',re.UNICODE)
    stop_words = stopwords.words('english')
    
    for textLine in textToTokenize:
        words = re.sub(usersPattern,'',textLine)
        words = re.sub(hashtagPattern,'',words)
        words = word_tokenize(words)
        for w in words:
            if w not in stop_words and w not in '@' and w not in '#':
                filteredSentence.append(w)
    return filteredSentence


def tweet_to_words(raw_tweet):
    tweet = ''.join(c for c in raw_tweet if c not in string.punctuation)
    tweet = re.sub('((www\S+)|(http\S+))', 'urlsite', tweet)
    tweet = re.sub(r'\d+', 'contnum', tweet)
    tweet = re.sub(' +',' ', tweet)
    words = tweet.lower().split()                             
    stops = set(stopwords.words("english"))
                 
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words ))
    
    
def users(tweet):
    user = []
    usersPattern = re.compile('@[a-zA-Z0-9]*',re.UNICODE)
    
    for t in tweet:
        u = re.findall(usersPattern,t)
        user.append(u)
    return user


def split_into_tokens(Text):
    return TextBlob(Text).words


def split_into_lemmas(Text):
    Text = Text.lower()
    words = TextBlob(Text).words
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]

Cleaning tweets text

In [ ]:
df["cleaned_text"] = df["text"].apply(lambda tweet: clean_tweets(tweet))
df["cleaned_text"].head()

Removing Mising Values

In [ ]:
df["country"] = np.NaN
user_location = df['user_location'].fillna(value='').str.split(',')
user_location.head()

Preparing Countries of users Tweet

In [ ]:
lat = cities['lat'].fillna(value = '').values.tolist()
lng = cities['lng'].fillna(value = '').values.tolist()
country = cities['country'].fillna(value = '').values.tolist()

# Getting all alpha 3 codes into  a list
world_city_iso3 = []
for c in cities['iso3'].str.lower().str.strip().values.tolist():
    if c not in world_city_iso3:
        world_city_iso3.append(c)
        
# Getting all alpha 2 codes into  a list    
world_city_iso2 = []
for c in cities['iso2'].str.lower().str.strip().values.tolist():
    if c not in world_city_iso2:
        world_city_iso2.append(c)
        
# Getting all countries into  a list        
world_city_country = []
for c in cities['country'].str.lower().str.strip().values.tolist():
    if c not in world_city_country:
        world_city_country.append(c)

# Getting all amdin names into  a list
world_states = []
for c in cities['admin_name'].str.lower().str.strip().tolist():
    world_states.append(c)


# Getting all cities into  a list
world_city = cities['city'].fillna(value = '').str.lower().str.strip().values.tolist()
world_city[:10]

In [ ]:
for each_loc in range(len(user_location)):
    ind = each_loc
    each_loc = user_location[each_loc]
    
    for each in each_loc:
        each = each.lower().strip()
        if each in world_city:
            order = world_city.index(each)
            df['country'][ind] = country[order].lower()
            continue
        if each in world_states:
            order= world_states.index(each)
            df['country'][ind] = country[order].lower()
            continue
        if each in world_city_country:
            order = world_city_country.index(each)
            df['country'][ind] = world_city_country[order].lower()
            continue
        if each in world_city_iso2:
            order = world_city_iso2.index(each)
            df['country'][ind] = world_city_country[order].lower()
            continue
        if each in world_city_iso3:
            order = world_city_iso3.index(each)
            df['country'][ind] = world_city_country[order].lower()
            continue


In [ ]:
print(df.shape)
df.head()

In [ ]:
import datetime

print("Current date:", datetime.date.today())

# saving datafrme
df.to_csv("covid_tweets_{}.csv".format(datetime.date.today()), index=False)

### Valid Tweets

In [ ]:
print('Total Number of valid Tweets Available: ', df['country'].isnull().sum())

<a id="4"></a>

## Data Visualization

### Top 10 Countries with Most Tweets

In [ ]:
tweet_per_country = df['country'].str.lower().dropna()
tw = tweet_per_country.value_counts().rename_axis('Country').reset_index(name='Tweet Count')
print(tw)
plt.rcParams['figure.figsize'] = (15,10)
plt.title('Top 10 Countries with Most Tweets',fontsize=15)
sns.set_palette("husl")
ax = sns.barplot(y=tw['Country'].head(10),x=tw['Tweet Count'].head(10))

### 10 Countries with Least Tweets

In [ ]:
tweet_per_country = df['country'].str.lower().dropna()
tw = tweet_per_country.value_counts().rename_axis('Country').reset_index(name='Tweet Count')
print(tw)
plt.rcParams['figure.figsize'] = (15,10)
plt.title('10 Countries with Least Tweets',fontsize=15)
sns.set_palette("husl")
ax = sns.barplot(y=tw['Country'][-9:],x=tw['Tweet Count'][-9:])

### Min and Max Dates Between The Dataset

In [ ]:
print(df["date"].min())
print(df["date"].max())

### Top 15 Countries with Most Tweets Diffrent Representation

In [ ]:
country_graph_03 = px.bar(x='Tweet Count',y='Country',data_frame=tw[:15],color='Country')
country_graph_03.show()

Let's see percent of NaNs for every column. We will visualize only columns with at least 1 missed value.

In [ ]:
missed = pd.DataFrame()
missed['column'] = df.columns

missed['percent'] = [round(100* df[col].isnull().sum() / len(df), 2) for col in df.columns]
missed = missed.sort_values('percent')
missed = missed[missed['percent']>0]

fig = px.bar(
    missed, 
    x='percent', 
    y="column", 
    orientation='h', 
    title='Missed values percent for every column (percent > 0)', 
    height=400, 
    width=600
)

fig.show()

Let's see top 40 users by number of tweets.

In [ ]:
ds = df['user_name'].value_counts().reset_index()
ds.columns = ['user_name', 'tweets_count']
ds = ds.sort_values(['tweets_count'])

fig = px.bar(
    ds.tail(40), 
    x="tweets_count", 
    y="user_name", 
    orientation='h', 
    title='Top 40 users by number of tweets', 
    width=800, 
    height=800
)

fig.show()

In [ ]:
df = pd.merge(df, ds, on='user_name')

Let's see most popular users.

In [ ]:
data = df.sort_values('user_followers', ascending=False)
data = data.drop_duplicates(subset='user_name', keep="first")
data = data[['user_name', 'user_followers', 'tweets_count']]
data = data.sort_values('user_followers')

fig = px.bar(
    data.tail(40), 
    x="user_followers", 
    y="user_name", 
    color='tweets_count',
    orientation='h', 
    title='Top 40 users by number of followers', 
    width=800, 
    height=800
)

fig.show()

And most friendly users.

In [ ]:
data = df.sort_values('user_friends', ascending=False)
data = data.drop_duplicates(subset='user_name', keep="first")
data = data[['user_name', 'user_friends', 'tweets_count']]
data = data.sort_values('user_friends')

fig = px.bar(
    data.tail(40), 
    x="user_friends", 
    y="user_name", 
    color = 'tweets_count',
    orientation='h', 
    title='Top 40 users by number of friends', 
    width=800, 
    height=800
)

fig.show()

Let's see how coronavirus affect to new users creation.

In [ ]:
df['user_created'] = pd.to_datetime(df['user_created'])
df['year_created'] = df['user_created'].dt.year
data = df.drop_duplicates(subset='user_name', keep="first")
data = data[data['year_created']>1970]
data = data['year_created'].value_counts().reset_index()
data.columns = ['year', 'number']

fig = px.bar(
    data, 
    x="year", 
    y="number", 
    orientation='v', 
    title='User created year by year', 
    width=800, 
    height=600
)

fig.show()

As we can see from chart coronavirus increases the number of new twitter users.

In [ ]:
df.head()

Let's see top 40 most popular locations by the number of tweets.

In [ ]:
ds = df['user_location'].value_counts().reset_index()
ds.columns = ['user_location', 'count']
ds = ds[ds['user_location']!='NA']
ds = ds.sort_values(['count'])

fig = px.bar(
    ds.tail(40), 
    x="count", 
    y="user_location", 
    orientation='h', title='Top 40 user locations by number of tweets', 
    width=800, 
    height=800
)

fig.show()

And also we can see the pie plot for the full picture about users locations.

In [ ]:
def pie_count(data, field, percent_limit, title):
    
    data[field] = data[field].fillna('NA')
    data = data[field].value_counts().to_frame()

    total = data[field].sum()
    data['percentage'] = 100 * data[field]/total    

    percent_limit = percent_limit
    otherdata = data[data['percentage'] < percent_limit] 
    others = otherdata['percentage'].sum()  
    maindata = data[data['percentage'] >= percent_limit]

    data = maindata
    other_label = "Others(<" + str(percent_limit) + "% each)"
    data.loc[other_label] = pd.Series({field:otherdata[field].sum()}) 
    
    labels = data.index.tolist()   
    datavals = data[field].tolist()
    
    trace=go.Pie(labels=labels,values=datavals)

    layout = go.Layout(
        title = title,
        height=600,
        width=600
        )
    
    fig = go.Figure(data=[trace], layout=layout)
    iplot(fig)
    
pie_count(df, 'user_location', 0.5, 'Number of tweets per location')

Now it's time to check last one categorical feature - `source`. Lets see top 40 sources by the number of tweets.

In [ ]:
ds = df['source'].value_counts().reset_index()
ds.columns = ['source', 'count']
ds = ds.sort_values(['count'])

fig = px.bar(
    ds.tail(40), 
    x="count", 
    y="source", 
    orientation='h', 
    title='Top 40 user sources by number of tweets', 
    width=800, 
    height=800
)

fig.show()

### 3. Additional features analysis

Lets create new feature - `hashtags_count` that will show us how many hashtags in the current tweet.

In [ ]:
df['hashtags'] = df['hashtags'].fillna('[]')
df['hashtags_count'] = df['hashtags'].apply(lambda x: len(x))
df.loc[df['hashtags'] == '[]', 'hashtags_count'] = 0

df[['hashtags','hashtags_count']].head()

And see the values for new created column.

In [ ]:
df['hashtags_count'].describe()

In [ ]:
fig = px.scatter(
    df, 
    x=df['hashtags_count'], 
    y=df['tweets_count'], 
    height=700,
    width=700,
    title='Total number of tweets for users and number of hashtags in every tweet'
)

fig.show()

Distribution of new feature over the number of tweets is expected - a lot of tweets with few number of hashtags and few tweets with huge number of hashtags.

In [ ]:
ds = df['hashtags_count'].value_counts().reset_index()
ds.columns = ['hashtags_count', 'count']
ds = ds.sort_values(['count'])
ds['hashtags_count'] = ds['hashtags_count'].astype(str) + ' tags'

fig = px.bar(
    ds, 
    x="count", 
    y="hashtags_count", 
    orientation='h', 
    title='Distribution of number of hashtags in tweets', 
    width=800, 
    height=600
)

fig.show()

Now we will see top 40 users that like to use hashtags a little bit more than others. 

In [ ]:
ds = df[df['tweets_count']>10]
ds = ds.groupby(['user_name', 'tweets_count'])['hashtags_count'].mean().reset_index()
ds.columns = ['user', 'tweets_count', 'mean_count']
ds = ds.sort_values(['mean_count'])

fig = px.bar(
    ds.tail(40), 
    x="mean_count", 
    y="user", 
    color='tweets_count',
    orientation='h', 
    title='Top 40 users with higher mean number of hashtags (at least 10 tweets per user)', 
    width=800, 
    height=800
)

fig.show()

### Just split day and time into separate columns

In [ ]:
df['date'] = pd.to_datetime(df['date']) 
df = df.sort_values(['date'])
df['day'] = df['date'].astype(str).str.split(' ', expand=True)[0]
df['time'] = df['date'].astype(str).str.split(' ', expand=True)[1]
df.head()

### Number of unique users per day

In [ ]:
ds = df.groupby(['day', 'user_name'])['hashtags_count'].count().reset_index()
ds = ds.groupby(['day'])['user_name'].count().reset_index()
ds.columns = ['day', 'number_of_users']
ds['day'] = ds['day'].astype(str) + ':00:00:00'
fig = px.bar(
    ds, 
    x='day', 
    y="number_of_users", 
    orientation='v',
    title='Number of unique users per day', 
    width=800, 
    height=800
)
fig.show()

### Now we are going to check how many tweets were for every day in our dataset.

In [ ]:
ds = df['day'].value_counts().reset_index()
ds.columns = ['day', 'count']
ds = ds.sort_values('count')
ds['day'] = ds['day'].astype(str) + ':00:00:00'
fig = px.bar(
    ds, 
    x='count', 
    y="day", 
    orientation='h',
    title='Tweets distribution over days present in dataset', 
    width=800, 
    height=800
)
fig.show()

### Lets do the same but for hours

In [ ]:
df['hour'] = df['date'].dt.hour
ds = df['hour'].value_counts().reset_index()
ds.columns = ['hour', 'count']
ds['hour'] = 'Hour ' + ds['hour'].astype(str)
fig = px.bar(
    ds, 
    x="hour", 
    y="count", 
    orientation='v', 
    title='Tweets distribution over hours', 
    width=800
)
fig.show()

### Lets split hashtags into separate column.

In [ ]:
def split_hashtags(x): 
    return str(x).replace('[', '').replace(']', '').split(',')

tweets_df = df.copy()
tweets_df['hashtag'] = tweets_df['hashtags'].apply(lambda row : split_hashtags(row))
tweets_df = tweets_df.explode('hashtag')
tweets_df['hashtag'] = tweets_df['hashtag'].astype(str).str.lower().str.replace("'", '').str.replace(" ", '')
tweets_df.loc[tweets_df['hashtag']=='', 'hashtag'] = 'NO HASHTAG'
tweets_df.head()

### And show top 20 hashtags on tweets.

In [ ]:
ds = tweets_df['hashtag'].value_counts().reset_index()
ds.columns = ['hashtag', 'count']
ds = ds.sort_values(['count'])
fig = px.bar(
    ds.tail(20), 
    x="count", 
    y='hashtag', 
    orientation='h', 
    title='Top 20 hashtags', 
    width=800, 
    height=700
)
fig.show()

### Now we are going to calculate the length for every tweet in dataset.

In [ ]:
df['tweet_length'] = df['text'].str.len()

In [ ]:
fig = px.histogram(
    df, 
    x="tweet_length", 
    nbins=80, 
    title='Tweet length distribution', 
    width=800,
    height=700
)
fig.show()

In [ ]:
ds = df[df['tweets_count']>=10]
ds = ds.groupby(['user_name', 'tweets_count'])['tweet_length'].mean().reset_index()
ds.columns = ['user_name', 'tweets_count', 'mean_length']
ds = ds.sort_values(['mean_length'])
fig = px.bar(
    ds.tail(40), 
    x="mean_length", 
    y="user_name", 
    color='tweets_count',
    orientation='h', 
    title='Top 40 users with the longest average length of tweet (at least 10 tweets)', 
    width=800, 
    height=800
)
fig.show()

In [ ]:
ds = df[df['tweets_count']>=10]
ds = ds.groupby(['user_name', 'tweets_count'])['tweet_length'].mean().reset_index()
ds.columns = ['user_name', 'tweets_count', 'mean_length']
ds = ds.sort_values(['mean_length'])
fig = px.bar(
    ds.head(40), 
    x="mean_length", 
    y="user_name", 
    color='tweets_count',
    orientation='h', 
    title='Top 40 users with the shortest average length of tweet (at least 10 tweets)', 
    width=800, 
    height=800
)
fig.show()

<a id="5"></a>
## Tweets text analysis

#### Here we are going to check the `text` feature of the dataset.
#### Lets see general wordcloud for this column.

In [ ]:
def build_wordcloud(text_column, title):
    words = ' '.join(text_column.dropna().astype(str))
    cleaned_words = " ".join([word for word in words.split()
                                if 'http' not in word
                                    and not word.startswith('@')
                                    and word != 'RT'])

    wordcloud = WordCloud(
        background_color='white',
        stopwords=set(STOPWORDS), 
        max_words=50, 
        max_font_size=40, 
        random_state=666
    ).generate(cleaned_words)

    fig = plt.figure(1, figsize=(14,14))
    plt.axis('off')
    fig.suptitle(title, fontsize=16)
    fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [ ]:
build_wordcloud(df['cleaned_text'], 'Revalent words in tweets for all dataset')

### Lets see world clouds for top 3 users.

In [ ]:
user_name = ds.sort_values(['tweets_count'], ascending=False)['user_name'].tolist()[0]
test_df = df[df['user_name']==user_name]
build_wordcloud(test_df['cleaned_text'], 'Revalent words in tweets for {}'.format(user_name))

In [ ]:
user_name = ds.sort_values(['tweets_count'], ascending=False)['user_name'].tolist()[1]
test_df = df[df['user_name']==user_name]
build_wordcloud(test_df['cleaned_text'], 'Revalent words in tweets for {}'.format(user_name))

In [ ]:
user_name = ds.sort_values(['tweets_count'], ascending=False)['user_name'].tolist()[2]
test_df = df[df['user_name']==user_name]
build_wordcloud(test_df['cleaned_text'], 'Revalent words in tweets for {}'.format(user_name))

### Let's also visualize WordCloud for user's description.

In [ ]:
build_wordcloud(df['user_description'], 'Prevalent words in tweets for Blood Donors India')

<a id="6"></a>

## Sentiment analysis

### using Tfidf Vectorizer to get features and Kmeans clustering algotithm to split data into 3 clusters.

In [ ]:
# vec = TfidfVectorizer(stop_words="english")
# vec.fit(df['text'].values)
# features = vec.transform(df['text'].values)

# kmeans = KMeans(n_clusters=3, random_state=0)
# kmeans.fit(features)

# res = kmeans.predict(features)
# df['Cluster'] = res
# df.head()

In [ ]:
# df[df['Cluster'] == 0].head(10)['text'].tolist()

In [ ]:
# df[df['Cluster'] == 1].head(10)['text'].tolist()

In [ ]:
# df[df['Cluster'] == 2].head(10)['text'].tolist()

In [ ]:
# print('Number of samples for class 0: ', len(df[df['Cluster'] == 0]))
# print('Number of samples for class 1: ', len(df[df['Cluster'] == 1]))
# print('Number of samples for class 2: ', len(df[df['Cluster'] == 2]))

In [ ]:
# for i in range(3):
#     build_wordcloud(df[df['Cluster'] == i]['text'].tolist(), 'Wordcloud for cluster ' + str(i))

## We can see that cluster 0 and 1 contains more or less positive tweets, but cluster 2 contains tweets with information about new cases, reports and regions.

In [ ]:
# feature = df['Cluster']
# top_n_active_cities = feature.value_counts()
# print(top_n_active_cities)

# ncount = feature.shape[0]

# plt.figure(figsize=(12,5))
# chart = sns.countplot(feature, order=feature.value_counts().index)
# plt.xticks(
#     rotation=45, 
#     horizontalalignment='right',
#     fontweight='light',
#     fontsize='x-large' 
# )
# chart.set_title('Sentiment Analysis of Tweets', fontsize=15)
# chart.set_xlabel("Sentiment",fontsize=20)
# chart.set_ylabel("Counts (Number of Tweets)",fontsize=20)

# # Make twin axis
# chart2 = chart.twinx()

# # Switch so count axis is on right, frequency on left
# chart2.yaxis.tick_left()
# chart.yaxis.tick_right()

# # Also switch the labels over
# chart.yaxis.set_label_position('right')
# chart2.yaxis.set_label_position('left')

# chart2.set_ylabel('Frequency [%]',fontsize=20)

# for p in chart.patches:
#     x=p.get_bbox().get_points()[:,0]
#     y=p.get_bbox().get_points()[1,1]
#     chart.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
#             ha='center', va='bottom') # set the alignment of the text

# # Use a LinearLocator to ensure the correct number of ticks
# chart.yaxis.set_major_locator(ticker.LinearLocator(11))

# # Fix the frequency range to 0-100
# chart2.set_ylim(0,100)
# chart.set_ylim(0,ncount)

# # And use a MultipleLocator to ensure a tick spacing of 10
# chart2.yaxis.set_major_locator(ticker.MultipleLocator(10))

# plt.show()

## Sentiment Analysis of COVID-19 Tweets using TextBlob

In [ ]:
# get sentiment using TextBlob
df["sentiment"] = df["cleaned_text"].dropna().parallel_apply(get_tweet_sentiment)

df["sentiment"].value_counts()

In [ ]:
# saving datafrme
df.to_csv("covid_tweets_with_sentiments_{}.csv".format(datetime.date.today()), index=False)

##### Sentiments of Tweets
The tweets having a positive, negative, or neutral sentiment have already been
determined. Here, the coding is done to display this information pictorially using
explode library, to make a pie chart to display this data. 

In [ ]:
import matplotlib.ticker as ticker

def show_sentiment_analysis_barplot(dataframe, sentiment_column, country="all"):
    
    if country != "all":
        dataframe = dataframe[dataframe.country == country]
        
    feature = dataframe[sentiment_column]
    top_n_active_cities = feature.value_counts()
    print(top_n_active_cities)

    ncount = feature.shape[0]

    plt.figure(figsize=(12,5))
    chart = sns.countplot(feature, order=feature.value_counts().index)
    plt.xticks(
        rotation=45, 
        horizontalalignment='right',
        fontweight='light',
        fontsize='x-large' 
    )
    
    if country == "all":
        chart.set_title('Sentiment Analysis of {} Tweets in all countries'.format(ncount), fontsize=15)
    else:
        chart.set_title('Sentiment Analysis of {} Tweets in {}'.format(ncount,' '.join([name.capitalize() for name in country.split(" ")])), fontsize=15)
        
    chart.set_xlabel("Sentiment",fontsize=20)
    chart.set_ylabel("Counts (Number of Tweets)",fontsize=20)

    # Make twin axis
    chart2 = chart.twinx()

    # Switch so count axis is on right, frequency on left
    chart2.yaxis.tick_left()
    chart.yaxis.tick_right()

    # Also switch the labels over
    chart.yaxis.set_label_position('right')
    chart2.yaxis.set_label_position('left')

    chart2.set_ylabel('Frequency [%]',fontsize=20)

    for p in chart.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        chart.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
                ha='center', va='bottom') # set the alignment of the text

    # Use a LinearLocator to ensure the correct number of ticks
    chart.yaxis.set_major_locator(ticker.LinearLocator(11))

    # Fix the frequency range to 0-100
    chart2.set_ylim(0,100)
    chart.set_ylim(0,ncount)

    # And use a MultipleLocator to ensure a tick spacing of 10
    chart2.yaxis.set_major_locator(ticker.MultipleLocator(10))

    plt.show()
    

show_sentiment_analysis_barplot(dataframe=df, sentiment_column='sentiment')

## Animation with geographical distribution of tweets

### Here I am going to show approach how to use plotly world map to demonstrate geographical distribution of tweets.

In [ ]:
df['location'] = df['user_location'].str.split(',', expand=True)[1].str.lstrip().str.rstrip()
res = df.groupby(['location'])['cleaned_text'].count().reset_index()

In [ ]:
df.head()

In [ ]:
country_dict = {}
for c in countries:
    country_dict[c.name] = c.alpha3
    
res['alpha3'] = res['location']
res = res.replace({"alpha3": country_dict})


country_list =  tw['Country'].tolist()[:5]

res = res[
    (res['alpha3'] == 'USA') | 
    (res['location'].isin(country_list)) | 
    (res['location'] != res['alpha3'])
]

gbr = ['England', 'UK', 'London', 'United Kingdom']
us = ['United States', 'NY', 'CA', 'GA']

res = res[res['location'].notnull()]
res.loc[res['location'].isin(gbr), 'alpha3'] = 'GBR'
res.loc[res['location'].isin(us), 'alpha3'] = 'USA'
res.loc[res['alpha3'] == 'USA', 'location'] = 'USA'
res.loc[res['alpha3'] == 'GBR', 'location'] = 'United Kingdom'
plot = res.groupby(['location', 'alpha3'])['cleaned_text'].sum().reset_index()
plot.head()

In [ ]:
fig = px.choropleth(
    plot, 
    locations="alpha3",
    hover_name='location',
    color="cleaned_text",
    projection="natural earth",
    color_continuous_scale=px.colors.sequential.Plasma,
    title='Tweets from different countries for every day',
    width=800, 
    height=600
)
fig.show()

In [ ]:
res = df.groupby([ 'location', 'user_name'])['text'].count().reset_index()
res = res[['location', 'user_name']]
res['alpha3'] = res['location']
res = res.replace({"alpha3": country_dict})

country_list = tw['Country'].tolist()[:5]

res = res[
    (res['alpha3'] == 'USA') | 
    (res['location'].isin(country_list)) | 
    (res['location'] != res['alpha3'])
]

gbr = ['England', 'UK', 'London', 'United Kingdom']
us = ['United States', 'NY', 'CA', 'GA']

res = res[res['location'].notnull()]
res.loc[res['location'].isin(gbr), 'alpha3'] = 'GBR'
res.loc[res['location'].isin(us), 'alpha3'] = 'USA'
res.loc[res['alpha3'] == 'USA', 'location'] = 'USA'
res.loc[res['alpha3'] == 'GBR', 'location'] = 'United Kingdom'
plot = res.groupby(['location', 'alpha3'])['user_name'].count().reset_index()

In [ ]:
fig = px.choropleth(
    plot, 
    locations="alpha3",
    hover_name='location',
    color="user_name",
    projection="natural earth",
    color_continuous_scale=px.colors.sequential.Plasma,
    title='Numbers of active users for every day',
    width=800, 
    height=600
)
fig.show()

## Machine Learning Analysis

Splitting dataset into train and test sets

In [ ]:
df = df[df['cleaned_text'].notna()]
df['cleaned_text']= df['cleaned_text'].dropna().apply(lambda x: tweet_to_words(x))
train_df, test_df = train_test_split(df, test_size=0.33,random_state=0)

train_clean_tweets = train_df['cleaned_text'].tolist()
test_clean_tweets = test_df['cleaned_text'].tolist()

Get features for each set as matrix of words counts using CountVectorizer

In [ ]:
count_vectorizer = CountVectorizer(analyzer = "word")
train_features = count_vectorizer.fit_transform(train_clean_tweets)
test_features = count_vectorizer.transform(test_clean_tweets)

Create a list of classifiers

In [ ]:
Classifiers = [    
    RandomForestClassifier(n_estimators=200, bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_jobs= -1,
            oob_score=False, random_state=10),
    
    AdaBoostClassifier(n_estimators=100, random_state=10),
    
    MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
      
    tree.DecisionTreeClassifier(),
    
    ensemble.ExtraTreesClassifier(n_estimators=100,
                                  max_features= 50,
                                  criterion= 'entropy'),

    ensemble.GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.001,n_estimators=50, random_state=None, verbose = 0)]
    

Fitting and evaluating classifiers

In [ ]:
dense_features = train_features.toarray()
dense_test = test_features.toarray()

accuracies = []
models = []

for classifier in Classifiers:
    try:
        fit = classifier.fit(train_features, train_df['sentiment'])
        pred = fit.predict(test_features)
    except Exception:
        fit = classifier.fit(dense_features, train_df['sentiment'])
        pred = fit.predict(dense_test)

    accuracy = accuracy_score(pred,test_df['sentiment'])
    print("Classifier Name: {}".format(classifier.__class__.__name__))
    print('>> Accuracy : {}'.format(str(accuracy)))
    accuracies.append(accuracy)
    
    print(">> Classification report for classifier {}: \n {}".format(classifier,
                                                                     classification_report(test_df['sentiment'],
                                                                     pred)))
    cm = confusion_matrix(test_df['sentiment'], pred)
    print(">> Confusion matrix: \n{}".format(cm))
    models.append(classifier.__class__.__name__)
    print("=" * 110)

Ploting the Model Performances

In [ ]:
data = pd.DataFrame({"Model":models, "Accuracy":accuracies})

fig = px.bar(
    data, 
    x="Model",
    y="Accuracy", 
    orientation='v', 
    title='Accuracies of Models', 
    width=800, 
    height=600,
)

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

**Get features using CountVectorizer and Ngrams**

In [ ]:
count_vectorizer = CountVectorizer(analyzer="word", 
                                   ngram_range=(1,2), 
                                   min_df=3, 
                                   max_df=.95, 
                                   stop_words='english')
train_features = count_vectorizer.fit_transform(train_clean_tweets)
test_features = count_vectorizer.transform(test_clean_tweets)

Fitting and evaluating classifiers

In [ ]:
dense_features = train_features.toarray()
dense_test = test_features.toarray()

accuracies = []
models = []

for classifier in Classifiers:
    try:
        fit = classifier.fit(train_features, train_df['sentiment'])
        pred = fit.predict(test_features)
    except Exception:
        fit = classifier.fit(dense_features, train_df['sentiment'])
        pred = fit.predict(dense_test)

    accuracy = accuracy_score(pred,test_df['sentiment'])
    print("Classifier Name: {}".format(classifier.__class__.__name__))
    print('>> Accuracy : {}'.format(str(accuracy)))
    accuracies.append(accuracy)
    
    print(">> Classification report for classifier {}: \n {}".format(classifier,
                                                                     classification_report(test_df['sentiment'],
                                                                     pred)))
    cm = confusion_matrix(test_df['sentiment'], pred)
    print(">> Confusion matrix: \n{}".format(cm))
    models.append(classifier.__class__.__name__)
    print("=" * 110)

Ploting the Model Performances after using Ngram

In [ ]:
data = pd.DataFrame({"Model":models, "Accuracy":accuracies})

fig = px.bar(
    data, 
    x="Model",
    y="Accuracy", 
    orientation='v', 
    title='Accuracies of Models using Ngrams', 
    width=800, 
    height=600,
)

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

### Sentiments Analysis of Specific Country

In [ ]:
df = pd.read_csv("covid_tweets_with_sentiments_{}.csv".format(datetime.date.today()))
print(df.shape)
df.head()

In [ ]:
df.columns

Build a module for text standardization

In [ ]:
# get topN similar sentences for a given sentence using Cosine Similarity
def most_similar_sentence(sentence, all_sentences):

#     print("Given sentence: ",sentence)
    max_cosine_sim = 0.5
    most_sim_sentence = ''
    most_sim_sentence_index = 0
    
    for index, sim_sentence in enumerate(all_sentences):
        cosine = Cosine(2)
        s0 = sentence
        s1 = sim_sentence
        p0 = cosine.get_profile(s0)
        p1 = cosine.get_profile(s1)
        if p1 and cosine.similarity_profiles(p0, p1) > max_cosine_sim:
            max_cosine_sim = cosine.similarity_profiles(p0, p1)
            most_sim_sentence = sim_sentence
            most_sim_sentence_index = index
            
    sentences_cosine_similarities = (s0,most_sim_sentence,max_cosine_sim)
#     print("Most similar sentence: {}".format(most_sim_sentence).encode('utf-8').strip())

    return (most_sim_sentence, max_cosine_sim, most_sim_sentence_index)

In [ ]:
country = "India"

sim_country = most_similar_sentence(country, df.country.dropna().unique().tolist())[0]
show_sentiment_analysis_barplot(dataframe=df, sentiment_column='sentiment', country=sim_country)

In [ ]:
build_wordcloud(df[df['country'] == sim_country]['cleaned_text'],
                'Wordcloud for {} Tweets'.format(' '.join([name.capitalize() for name in sim_country.split(" ")])))

In [ ]:
build_wordcloud(df[(df['country'] == sim_country) & (df['sentiment'] == 'positive')]['cleaned_text'],
                'Wordcloud for {} Positive Tweets'.format(' '.join([name.capitalize() for name in sim_country.split(" ")])))

In [ ]:
build_wordcloud(df[(df['country'] == sim_country) & (df['sentiment'] == 'negative')]['cleaned_text'],
                'Wordcloud for {} Negative Tweets'.format(' '.join([name.capitalize() for name in sim_country.split(" ")])))

In [ ]:
build_wordcloud(df[(df['country'] == sim_country) & (df['sentiment'] == 'neutral')]['cleaned_text'],
                'Wordcloud for {} Neutral Tweets'.format(' '.join([name.capitalize() for name in sim_country.split(" ")])))

#### Get covid daily cases 

In [ ]:
import covid_daily

covid_daily_df = covid_daily.overview(as_json=False)

covid_daily_df.head()

#### APIs for covid-19 cases stats. per country 

In [ ]:
def all_world_population_countries_names():
    url = "https://world-population.p.rapidapi.com/allcountriesname"
    headers = {
        'x-rapidapi-key': "134117ae79msh40bb2931f9c7e4ap1aa445jsn8d1982670b09",
        'x-rapidapi-host': "world-population.p.rapidapi.com"
        }
    response = requests.request("GET", url, headers=headers)
    return response.json() 


def get_world_population_per_country(country):
    url = "https://world-population.p.rapidapi.com/population"
    querystring = {"country_name":country}
    headers = {
        'x-rapidapi-key': "134117ae79msh40bb2931f9c7e4ap1aa445jsn8d1982670b09",
        'x-rapidapi-host': "world-population.p.rapidapi.com"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    return response.json()


def get_covid19api_all_countries():
    url = "https://api.covid19api.com/countries"
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload, timeout=10)
    return response.json()


def get_covid19api_stats_country(country , from_date , to_date ):
    url = "https://api.covid19api.com/total/country/{}?from={}&to={}".format(country , from_date , to_date )
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data = payload )
    return response


def get_covid_observer_countries():
    url = "https://covid.observer/us/#countries"
    response = requests.request("GET", url, timeout=10)
    soup = BeautifulSoup(response.content, "html.parser")
    countries_list_element = soup.find_all("div",{"class":"countries-list"})[2]
    
    result = {}
    for a_tag in countries_list_element.find_all("a"):
        result[a_tag.text] = "https://covid.observer" + a_tag['href']
    
    return result

#### Get covid cases dataframe for specific country

In [ ]:
# get world countries
all_countries = get_covid_observer_countries()

# get most similar country
sim_country = most_similar_sentence(country, all_countries.keys())[0]

# get API url for the similar country
url = all_countries[sim_country]

# make a get request
page = requests.get(url)

# declare a BeautifulSoup object from request content
soup = BeautifulSoup(page.text, 'lxml')

# text mining on BeautifulSoup object
table_data = soup.find('table')
headers = []
for i in table_data.find_all('th'):
    title = i.text
    headers.append(title)

# creatr a DataFrame
specific_country_stat_df = pd.DataFrame(columns = headers)

for j in table_data.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [tr.text for tr in row_data]
    length = len(specific_country_stat_df)
    specific_country_stat_df.loc[length] = row
        

specific_country_stat_df.columns = ['date','confirmed_cases','daily_growth','recovered_cases',
                                     'fatal_cases','active_cases','recovery_rate','mortality_rate',
                                     'affected_population','confirmed_per_1000','died_per_1000']

specific_country_stat_df['index'] = specific_country_stat_df.index

specific_country_stat_df['country'] = sim_country

specific_country_stat_df.head()

#### Preprocessing covid cases dataframe

In [ ]:
for col in ['confirmed_cases', 'recovered_cases', 'fatal_cases','active_cases']:
    specific_country_stat_df[col] = specific_country_stat_df[col].parallel_apply(lambda x : int(x.replace(",","")))
    specific_country_stat_df[col] = specific_country_stat_df[col].astype(int)

specific_country_stat_df.head()

Create a new column for the date with year

In [ ]:
y = [', 2021'] * specific_country_stat_df[specific_country_stat_df['date']=='Jan 1'].index[0]

observations_num_in_2020 = len(specific_country_stat_df) - specific_country_stat_df[specific_country_stat_df['date']=='Jan 1'].index[0]
print(observations_num_in_2020)

y.extend([', 2020'] * observations_num_in_2020)
len(y)

In [ ]:
specific_country_stat_df['year'] = y
specific_country_stat_df['date_with_year'] = specific_country_stat_df['date'] + specific_country_stat_df['year']
specific_country_stat_df['date_with_year'].head()

Show the beginning and end of dates

In [ ]:
print("Beginning: ", pd.to_datetime(specific_country_stat_df['date_with_year']).min())
print("End: ", pd.to_datetime(specific_country_stat_df['date_with_year']).max())

#### Get covid vaccinations

In [ ]:
# call API of covid vaccinations
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv"
page = requests.get(url)

# save data into csv file
vaccinations_file_path = 'vaccinations.csv'
vaccinations_file = open(vaccinations_file_path, 'wb')
vaccinations_file.write(page.content)
vaccinations_file.close()

# load csv as dataframe
covid_vaccinations_df = pd.read_csv(vaccinations_file_path)
covid_vaccinations_df.head()

Get covid vaccinations for specific country

In [ ]:
sim_country = most_similar_sentence(specific_country_stat_df['country'].iloc[0], covid_vaccinations_df['location'].unique())[0]
specific_covid_vaccinations_df = covid_vaccinations_df[covid_vaccinations_df.location == sim_country]
specific_covid_vaccinations_df.head()

Show the beginning and end of dates

In [ ]:
print("Beginning: ", pd.to_datetime(specific_covid_vaccinations_df['date']).min())
print("End: ", pd.to_datetime(specific_covid_vaccinations_df['date']).max())

In [ ]:
# Parsing dates to datetime
specific_covid_vaccinations_df['date'] = pd.to_datetime(specific_covid_vaccinations_df['date'])
specific_country_stat_df['date_with_year'] = pd.to_datetime(specific_country_stat_df['date_with_year'])

#### Merging covid cases and vaccinations dataframes

In [ ]:
specific_country_stat_vac_df = pd.merge(specific_covid_vaccinations_df,
        specific_country_stat_df, 
        how='right',
        left_on=["date"],
        right_on=["date_with_year"])

specific_country_stat_vac_df.head()

#### Create a dataframe for sentiments counts

In [ ]:
# filter tweets df by specific country
specific_country_tweets_df = df[df.country == most_similar_sentence(country, df['country'].dropna().unique())[0]]

# create a dataframe of groubing by sentiments counts per day
sentiments_counts_df = pd.DataFrame({'count' : specific_country_tweets_df.groupby(['day','sentiment'])['sentiment'].count()}).reset_index()

# create a pivot table for sentiments counts
sentiments_counts_df = sentiments_counts_df.pivot(index="day", columns="sentiment", values="count").reset_index()
sentiments_counts_df.head()

In [ ]:
# Make cumulative sum for sentiments counts
for col in ['negative','neutral','positive']:
    sentiments_counts_df[col] = sentiments_counts_df[col].cumsum(axis = 0)

sentiments_counts_df.head()

In [ ]:
# Parse date of sentiments_counts_df to datetime
sentiments_counts_df['day'] = pd.to_datetime(sentiments_counts_df['day'])

In [ ]:
print("Number of intersected dates between specific_country_stat_vac_df and sentiments_counts_df: ",
      len(set(specific_country_stat_vac_df['date_with_year'].unique()).intersection(set(sentiments_counts_df['day'].unique()))))

#### Merging all dataframes for specific country (sentiments counts, covid cases and vaccinations stats) 

In [ ]:
specific_country_sentiment_stat_vac_df = pd.merge(sentiments_counts_df,
                                                  specific_country_stat_vac_df, 
                                                  how='right',
                                                  left_on=["day"],
                                                  right_on=["date_with_year"])
specific_country_sentiment_stat_vac_df.head()

In [ ]:
specific_country_sentiment_stat_vac_df.columns

Filter dataframe to have only rows whose date is in tweets dataframe

In [ ]:
specific_country_sentiment_stat_vac_df = specific_country_sentiment_stat_vac_df[specific_country_sentiment_stat_vac_df.date_with_year.isin(sentiments_counts_df.day)]
specific_country_sentiment_stat_vac_df.head()

#### Scaling numerical columns for data visualization

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()

for col in ['negative','neutral','positive','confirmed_cases', 
            'recovered_cases', 'fatal_cases', 'active_cases',
            'total_vaccinations', 'people_fully_vaccinated']:
    specific_country_sentiment_stat_vac_df[col] = min_max_scaler.fit_transform(specific_country_sentiment_stat_vac_df[[col]].values)

specific_country_sentiment_stat_vac_df.head()

#### Draw a Multiple lines Plot 

In [ ]:
base_stats_fig = go.Figure()

for column in ['confirmed_cases', 'recovered_cases', 'fatal_cases', 'active_cases',
               'total_vaccinations', 'people_fully_vaccinated',
              'negative', 'neutral', 'positive']:
    color_dict = {
        "confirmed_cases": "#118ab2",
        "recovered_cases": "#ef476f",
        "fatal_cases": "#06d6a0",
        "active_cases": "#073b4c",
        "total_vaccinations": " #979a9a",
        "people_fully_vaccinated": " #979a9a",
        "negative": " #641e16",
        "neutral": " #641e16",
        "positive": " #641e16",
        }
    base_stats_fig.add_trace(
        go.Scatter(
            x = specific_country_sentiment_stat_vac_df['date_with_year'],
            y = specific_country_sentiment_stat_vac_df[column],
            name = column,
            line = dict(color=color_dict[column]),
            hovertemplate ='<br><b>Date</b>: %{x}'+'<br><i>Count</i>:'+'%{y}',
        )
    )
    

base_stats_fig.update_layout(
    updatemenus=[
        dict(
        buttons=list(
            [dict(label = 'All Cases',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True, True, True, True, True, True, True]},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             
             dict(label = 'active_cases',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False, False, False, False, False, False, False]},
                          {'title': 'active_cases',
                           'showlegend':True}]),
             
             dict(label = 'confirmed_cases',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False, False, False, False, False, False, False, False]},
                          {'title': 'confirmed_cases',
                           'showlegend':True}]),
             
             dict(label = 'recovered_cases',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, False, False, False, False, False, False, False]},
                          {'title': 'recovered_cases',
                           'showlegend':True}]),

             dict(label = 'fatal_cases',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, False, False, False, False, False, False]},
                          {'title': 'fatal_cases',
                           'showlegend':True}]),
             
             dict(label = 'total_vaccinations',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False, False, False, False, False, False, False]},
                          {'title': 'total_vaccinations',
                           'showlegend':True}]),

             dict(label = 'people_fully_vaccinated',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, True, False, False, False, False, False, False]},
                          {'title': 'people_fully_vaccinated',
                           'showlegend':True}]),
                          
             dict(label = 'negative',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, True, False, False, False, False, False]},
                          {'title': 'negative',
                           'showlegend':True}]),
             
             dict(label = 'neutral',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, True, False, False, False, False]},
                          {'title': 'neutral',
                           'showlegend':True}]),

             dict(label = 'positive',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, False, True, False, False, False]},
                          {'title': 'positive',
                           'showlegend':True}]),

            ]),
             type = "dropdown",
             direction="down",
             showactive=True,
             x=0,
             xanchor="left",
             y=1.25,
             yanchor="top"
        ),
        dict(
        buttons=list(
            [dict(label = 'Linear Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'linear'}},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Log Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'log'}},
                          {'title': 'active_cases',
                           'showlegend':True}]),
            ]),
             type = "dropdown",
             direction="down",
             showactive=True,
             x=0,
             xanchor="left",
             y=1.36,
             yanchor="top"
        )
    ])

base_stats_fig.update_xaxes(showticklabels=False)
base_stats_fig.update_layout(
    title_text="Statistics about Covid19 in {}".format(country), title_x=0.5, title_font_size=20,
    legend=dict(orientation='h', yanchor='top', y=1.15, xanchor='right', x=1), 
    paper_bgcolor="mintcream",
    xaxis_title="Date",
    yaxis_title="# of Cases")

base_stats_fig.show()